In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
  # You can change this to the appropriate encoding
df = pd.read_csv("/content/SIH_DataSet.csv", encoding='latin-1',usecols=['TITLE','TEXT','LABEL'])
df.head(5)
df2=df.iloc[1,1]

In [ ]:
df=df.iloc[:15,-3:]
df.columns=['title','text','label']
df.head()
first_row_text = df.loc[0, 'text']

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_list = set(stopwords.words('english'))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
lm = WordNetLemmatizer()
import re

corpus = []

# 'title' column contains only string values
df['title'] = df['title'].apply(lambda x: str(x))

for i in range(len(df)):
    try:
        # Apply the modified regular expression pattern
        review = re.sub('[^a-zA-Z0-9\s]', '', df['title'][i])
        review = review.lower()
        review = review.split()
        review = [lm.lemmatize(x) for x in review if x not in stopwords_list]
        review = " ".join(review)
        corpus.append(review)
    except Exception as e:
        #exceptions that may occur during processing
        print(f"Error processing row {i}: {e}")

print(type(corpus), len(corpus))




<class 'list'> 15


In [ ]:
tf = TfidfVectorizer()
x = tf.fit_transform(corpus).toarray()
y = df['label']
len(y)

15

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 10, stratify = y )
df['ColumnName'] = df['label'].astype(str)
print(y_train,type(y_train))

13     True
14    False
4      True
12     True
7     False
9     False
0      True
2     False
5     False
10     True
Name: label, dtype: object <class 'pandas.core.series.Series'>


In [ ]:
label_encoder = pd.factorize(y_train)
y_train_encoded = label_encoder[0]

# Check unique labels and data type of y_train_encoded
unique_labels = np.unique(y_train_encoded)
print("Unique labels:", unique_labels)
print("Data type of y_train_encoded:", type(y_train_encoded))

# Create a RandomForestClassifier
rf = RandomForestClassifier()

# Fit the classifier to your training data
rf.fit(x_train, y_train_encoded)

Unique labels: [0 1]
Data type of y_train_encoded: <class 'numpy.ndarray'>


RandomForestClassifier()

In [ ]:
class Evaluation:

    def __init__(self,model,x_train,x_test,y_train,y_test):
        self.model = model
        self.x_train = x_train
        self.x_test = x_test
        self.y_train = y_train
        self.y_test = y_test

    def train_evaluation(self):
        y_pred_train = self.model.predict(self.x_train)

        acc_scr_train = accuracy_score(self.y_train,y_pred_train)
        print("Accuracy Score On Training Data Set :",acc_scr_train)
        print()

        con_mat_train = confusion_matrix(self.y_train,y_pred_train)
        print("Confusion Matrix On Training Data Set :\n",con_mat_train)
        print()

        class_rep_train = classification_report(self.y_train,y_pred_train)
        print("Classification Report On Training Data Set :\n",class_rep_train)


    def test_evaluation(self):
        y_pred_test = self.model.predict(self.x_test)

        acc_scr_test = accuracy_score(self.y_test,y_pred_test)
        print("Accuracy Score On Testing Data Set :",acc_scr_train)
        print()

        con_mat_test = confusion_matrix(self.y_test,y_pred_test)
        print("Confusion Matrix On Testing Data Set :\n",con_mat_train)
        print()

        class_rep_test = classification_report(self.y_test,y_pred_test)
        print("Classification Report On Testing Data Set :\n",class_rep_train)



In [ ]:
class Preprocessing:

    def __init__(self,data):
        self.data = data

    def text_preprocessing_user(self):
      lm = WordNetLemmatizer()
      import re

      corpus = []

      # Ensure that 'title' column contains only string values
      df['title'] = df['title'].apply(lambda x: str(x))

      for i in range(len(df)):
          try:
              # Apply the modified regular expression pattern
              review = re.sub('[^a-zA-Z0-9\s]', '', df['title'][i])
              review = review.lower()
              review = review.split()
              review = [lm.lemmatize(x) for x in review if x not in stopwords_list]
              review = " ".join(review)
              corpus.append(review)
          except Exception as e:
              # Handle any other exceptions that may occur during processing
              print(f"Error processing row {i}: {e}")
      return corpus

In [ ]:
class Prediction:

    def __init__(self,pred_data, model):
        self.pred_data = pred_data
        self.model = model

    def prediction_model(self):
        preprocess_data = Preprocessing(self.pred_data).text_preprocessing_user()
        data = tf.transform(preprocess_data)
        print(tf)
        prediction = self.model.predict(data)
        print(prediction)
        if prediction [0] == 0 :
            return "The News Is Fake"

        else:
            return "The News Is Real"

In [ ]:
data = first_row_text
Prediction(data,rf).prediction_model()

TfidfVectorizer()
[0 0 1 1 0 1 1 1 0 1 0 0 0 0 1]


'The News Is Fake'

In [ ]:
user_data = '15 Civilians Airstrike Have Been Identified'
Prediction(user_data,rf).prediction_model()

TfidfVectorizer()
[0 0 1 1 0 1 1 1 0 1 0 0 0 0 1]


'The News Is Fake'